<a href="https://colab.research.google.com/github/christee8/lsa-model/blob/master/question_answering_english_bert_squad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install transformers

In [71]:
import torch
import textwrap
import time
import numpy as np
from transformers import BertForQuestionAnswering, BertTokenizer

In [72]:
# https://huggingface.co/models?search=squad2

model = BertForQuestionAnswering.from_pretrained('deepset/bert-large-uncased-whole-word-masking-squad2')
tokenizer = BertTokenizer.from_pretrained('deepset/bert-large-uncased-whole-word-masking-squad2')

# model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')
# tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')

In [73]:
def decode_answer(start_idx, end_idx, tokens):
    if tokens[start_idx] == '[CLS]' or tokens[end_idx] == '[CLS]' or tokens[start_idx] == '[SEP]':
      return "<No Answer>"

    answer = ""

    for idx in range(start_idx, end_idx + 1):
      if tokens[idx] == '[SEP]':
        break
      elif tokens[idx][0:2] == '##':
          answer += tokens[idx][2:]
      else:
          if idx != start_idx:
            answer += ' '
          answer += tokens[idx]

    return answer

In [74]:
def get_answer(start_scores, end_scores, input_ids):
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Invalid range of answer span
    if end_idx < start_idx:
      scores = []

      for start_idx in range(start_scores.shape[1]):
        for end_idx in range(start_idx, start_scores.shape[1]):
          score = start_scores[0][start_idx] + end_scores[0][end_idx]
          scores.append({'start_idx': start_idx, 'end_idx': end_idx, 'score': score})
  
      # Select answer span with highest score
      best = sorted(scores, key=lambda i: i['score'], reverse=True)[0]
      start_idx = best['start_idx']
      end_idx = best['end_idx']

    return decode_answer(start_idx, end_idx, tokens)

In [75]:
def answer_question(question, answer_text):
    '''
    Takes a question string and an answer_text string,
    finds answer to the question string in the answer_text string
    and prints it.
    '''

    # Tokenize input
    encoded_dict = tokenizer(question, answer_text)
    input_ids = encoded_dict['input_ids']
    token_type_ids = encoded_dict['token_type_ids']

    # Evaluate model
    with torch.no_grad():
      start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

    # Get answer
    answer = get_answer(start_scores, end_scores, input_ids)

    return answer

In [76]:
# https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/Packet_switching.html?model=nlnet%20(single%20model)%20(Microsoft%20Research%20Asia)&version=v2.0
answer_text = "Starting in the late 1950s, American computer scientist Paul Baran developed the concept Distributed Adaptive Message Block Switching with the goal to provide a fault-tolerant, efficient routing method for telecommunication messages as part of a research program at the RAND Corporation, funded by the US Department of Defense. This concept contrasted and contradicted the theretofore established principles of pre-allocation of network bandwidth, largely fortified by the development of telecommunications in the Bell System. The new concept found little resonance among network implementers until the independent work of Donald Davies at the National Physical Laboratory (United Kingdom) (NPL) in the late 1960s. Davies is credited with coining the modern name packet switching and inspiring numerous packet switching networks in Europe in the decade following, including the incorporation of the concept in the early ARPANET in the United States."

wrapper = textwrap.TextWrapper(width=80)
print(wrapper.fill(answer_text))

Starting in the late 1950s, American computer scientist Paul Baran developed the
concept Distributed Adaptive Message Block Switching with the goal to provide a
fault-tolerant, efficient routing method for telecommunication messages as part
of a research program at the RAND Corporation, funded by the US Department of
Defense. This concept contrasted and contradicted the theretofore established
principles of pre-allocation of network bandwidth, largely fortified by the
development of telecommunications in the Bell System. The new concept found
little resonance among network implementers until the independent work of Donald
Davies at the National Physical Laboratory (United Kingdom) (NPL) in the late
1960s. Davies is credited with coining the modern name packet switching and
inspiring numerous packet switching networks in Europe in the decade following,
including the incorporation of the concept in the early ARPANET in the United
States.


In [77]:
question = "What did Paul Baran develop?"
print(question)

# Ground Truth Answers:
# Paul Baran developed the concept Distributed Adaptive Message Block Switching
# the concept Distributed Adaptive Message Block Switching
# Distributed Adaptive Message Block Switching
answer = answer_question(question, answer_text)
print("Answer:", answer)

What did Paul Baran develop?
Answer: distributed adaptive message block switching


In [78]:
question = "What did Distributed Adaptive Message Block Switching do?"
print(question)

# Ground Truth Answers:
# provide a fault-tolerant, efficient routing method for telecommunication messages
# fault-tolerant, efficient routing method
answer = answer_question(question, answer_text)
print("Answer:", answer)

What did Distributed Adaptive Message Block Switching do?
Answer: provide a fault - tolerant , efficient routing method for telecommunication messages


In [79]:
question = "In the late 1950s what concept was developed?"
print(question)

# Ground Truth Answers:
# <No Answer>
answer = answer_question(question, answer_text)
print("Answer:", answer)

In the late 1950s what concept was developed?
Answer: distributed adaptive message block switching


In [80]:
question = "How much funding did the RAND Corporation receive?"
print(question)

# Ground Truth Answers:
# <No Answer>
answer = answer_question(question, answer_text)
print("Answer:", answer)

How much funding did the RAND Corporation receive?
Answer: us department of defense


In [81]:
question = "What was the independent work of Donald Davis?"
print(question)

# Ground Truth Answers:
# <No Answer>
answer = answer_question(question, answer_text)
print("Answer:", answer)

What was the independent work of Donald Davis?
Answer: <No Answer>


In [82]:
answer_text = "Cat has 4 legs."

In [83]:
question = "How many eyes does cat have?"
print(question)

answer = answer_question(question, answer_text)
print("Answer:", answer)

How many eyes does cat have?
Answer: <No Answer>


In [84]:
question = "How many legs does cat have?"
print(question)

answer = answer_question(question, answer_text)
print("Answer:", answer)

How many legs does cat have?
Answer: 4
